In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import sys; sys.path.insert(0, '../')
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import mne 
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
pp = dict(surface='white', hemi='both')

ImportError: cannot import name 'pad_sequences' from 'keras.preprocessing.sequence' (c:\Users\Lukas\Envs\invertenv\lib\site-packages\keras\preprocessing\sequence.py)

# Get Forward Model

In [2]:
info = get_info(kind='biosemi32')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    2.6s remaining:    4.4s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    2.6s remaining:    1.5s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.9s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.2s remaining:    0.4s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.3s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.4s remaining:    0.7s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.4s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.4s finished


# Get sample data

In [20]:
# settings = dict(number_of_sources=1, extents=40, duration_of_trial=0.01, target_snr=99999999999)
settings = dict(number_of_sources=3, extents=(1, 40), duration_of_trial=1, target_snr=2)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()
M = evoked.data

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

-- number of adjacent vertices : 5124
Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 58.81it/s]


source data shape:  (5124, 1000) (5124, 1000)


100%|██████████| 2/2 [00:00<00:00, 18.18it/s]

Using control points [3.27471165e-11 1.81211507e-10 3.33690064e-08]


For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Regularization Optimizations

In [15]:
def make_sloreta_inverse_operator(leadfield, alpha=0.001, noise_cov=None):
    """ Calculate the inverse operator using standardized Low Resolution
    TomogrAphy (sLORETA).
    

    Parameters
    ----------
    leadfield : numpy.ndarray
        Leadfield (or gain matrix) G which constitutes the forward model of M =
        J @ G, where sources J are projected through the leadfield producing the
        observed EEG matrix M.
    adjacency : numpy.ndarray
        The source adjacency matrix (n_dipoles x n_dipoles) which represents the
        connections of the dipole mesh.
    alpha : float
        The regularization parameter.

    Return
    ------
    inverse_operator : numpy.ndarray
        The inverse operator that is used to calculate source.

    """
    n_chans, _ = leadfield.shape
    if noise_cov is None:
        noise_cov = np.identity(n_chans)
        
    if alpha.lower() == 'auto':
        eigenvals = np.linalg.eig(leadfield @leadfield.T)[0]
        alpha = np.max(eigenvals) / 2e4
        alphas = [alpha*r for r in range(12)]
    else:
        alphas = [alpha,]
    # create one inverse operator for each level of regularization
    inverse_operators = []
    for alpha in alphas:
        K_MNE = leadfield.T @ np.linalg.inv(leadfield @ leadfield.T + alpha * noise_cov)
        W_diag = 1 / np.diag(K_MNE @ leadfield)
        W_slor = np.sqrt(np.diag(W_diag))
        
        inverse_operator = W_slor @ K_MNE
        inverse_operators.append( inverse_operator )
    return inverse_operators


class InverseOperator:
    ''' This class holds the inverse operator, which may be a simple
    numpy.ndarray matrix or some object like an esinet.net()
    '''
    def __init__(self, inverse_operator, solver_name):
        self.solver_name = solver_name
        self.data = inverse_operator
        self.handle_inverse_operator()

        self.has_multiple_operators()
    def has_multiple_operators(self):
        ''' Check if there are multiple inverse_operators.'''
        if type(self.data) == list:
            if len(self.data) > 1:
                return True
        return False
    def handle_inverse_operator(self, inverse_operator):
        if type(self.data) != list:
            self.data = [self.data,]
        self.type = type(self.data[0])
        
    
    def apply(self, evoked, forward, verbose=0):
        ''' Apply the inverse operator to the evoked object to calculate the source.
    
        Parameters
        ----------
        evoked : mne.EvokedArray
            The evoked object containing evoked M/EEG data.
        
        Return
        ------
        stc : mne.SourceEstimate
            The source estimate object containing the source
        '''
        # Do some preprocessing/ whitening?
        M = evoked.data
        assert type(self.data) == list, "self.data is not a list so something is terribly wrong"

        if len(self.data) > 1:
            # Do the L corner thingy
        else:
            if self.type == np.ndarray:
                inverse_operator.data @ M 


        elif inverse_operator.type == list:
            maximum_a_posteriori, A, S = inverse_operator.data
            # transform data M with spatial (A) and temporal (S) projector
            M_ = A @ M @ S
            # invert transformed data M_ to tansformed sources J_
            J_ = maximum_a_posteriori @ M_
            # Project tansformed sources J_ back to original time frame using temporal projector S
            source_mat =  J_ @ S.T 
        elif inverse_operator.type == esinet.Net:
            source_mat = inverse_operator.data.predict(evoked)[0].data
        else:
            msg = f"type of inverse operator ({inverse_operator.type}) unknown"
            raise AttributeError(msg)
        
        # Convert source to mne.SourceEstimate object
        source_model = forward['src']
        vertices = [source_model[0]['vertno'], source_model[1]['vertno']]
        tmin = evoked.tmin
        sfreq = evoked.info["sfreq"]
        tstep = 1/sfreq
        subject = evoked.info["subject_info"]

        if subject is None:
            subject = "fsaverage"
        
        stc = mne.SourceEstimate(source_mat, vertices, tmin=tmin, tstep=tstep, subject=subject, verbose=verbose)
        return stc


In [21]:
from invert import make_inverse_operator, apply_inverse_operator, InverseOperator
from invert.evaluate import nmse, corr
# from invert.solvers.loreta import make_sloreta_inverse_operator
from tqdm.notebook import tqdm
solver = "slor"
alpha = 1/settings["target_snr"]**2

# inverse_operator = make_inverse_operator(fwd, solver=solver, evoked=evoked, alpha=alpha, verbose=0)
inverse_operator = make_sloreta_inverse_operator(leadfield, alpha='auto')
inverse_operator = InverseOperator(inverse_operator, solver)
stc_hat = apply_inverse_operator(evoked, inverse_operator, fwd)
stc_hat.plot(**pp)



Using control points [3.29159778e-08 3.71957356e-08 7.63551158e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [2.71529166e-08 3.01263004e-08 4.82658874e-08]
Using control points [2.64545841e-11 1.36834743e-10 1.57361275e-08]


In [54]:
solver = "dSPM"
# alpha_heuristic = 1/settings["target_snr"]**2
alpha = 2000
inverse_operator = make_inverse_operator(fwd, solver=solver, evoked=evoked, alpha=alpha, verbose=0)
stc_hat = apply_inverse_operator(evoked, inverse_operator, fwd)
stc_hat.plot(**pp)

Using control points [6.52956037e-05 7.85430437e-05 1.65699895e-04]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [4.54311450e-05 5.30130690e-05 7.36848788e-05]
Using control points [4.54311450e-05 5.30130690e-05 7.36848788e-05]


# Tests

In [ ]:
from invert import make_inverse_operator, apply_inverse_operator
from invert.adapters import contextualize_bd
from invert.evaluate import nmse

solver = "lstm"
# inverse_operator = make_inverse_operator(fwd, solver=solver, evoked=evoked, verbose=1)

stc_hat = apply_inverse_operator(evoked, inverse_operator, fwd)
error = np.median(nmse(stc.data, stc_hat.data))
print(f"error={error:.4f}")
stc_hat.plot(**pp, brain_kwargs=dict(title=solver))

# stc_hat.data = contextualize_bd(stc_hat.data, leadfield, num_epochs=20)
# error = np.median(nmse(stc.data, stc_hat.data))
# print(f"error={error:.4f}")
# stc_hat.plot(**pp, brain_kwargs=dict(title=solver + " contextualized"))


1/1 [==============================] - 1s 1s/step


c:\Users\lukas\virtualenvs\invertenv\lib\site-packages\scipy\optimize\_minimize.py:863: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


error=0.0260
Using control points [2.69993792e-09 3.79865658e-09 1.90327871e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [1.33156213e-09 2.41506867e-09 3.25942281e-08]
Using control points [2.45756571e-09 3.58205804e-09 7.57540420e-09]


In [ ]:
from invert import all_solvers, make_inverse_operator, apply_inverse_operator
from invert.adapters import contextualize_bd
from invert.evaluate import nmse, corr
stc.plot(**pp, brain_kwargs=dict(title="Ground Truth"))
errors = dict()
for solver in all_solvers:
    print(solver)
    inverse_operator = make_inverse_operator(fwd, solver=solver, evoked=evoked, alpha=1/4.5)
    stc_hat = apply_inverse_operator(evoked, inverse_operator, fwd)
    # stc_hat.plot(**pp, brain_kwargs=dict(title=solver))
    errors[solver] = corr(stc.data, stc_hat.data)
    
solver = "esinet"
stc_hat = net.predict(evoked)[0]
# stc_hat.plot(**pp, brain_kwargs=dict(title=solver))
errors[solver] = corr(stc.data, stc_hat.data)

solver += " contextualized"
stc_hat.data = contextualize_bd(stc_hat.data, leadfield)
errors[solver] = corr(stc.data, stc_hat.data)
# stc_hat.plot(**pp, brain_kwargs=dict(title=solver))

In [ ]:
solver = "LORETA"
inverse_operator = make_inverse_operator(fwd, solver=solver, evoked=evoked, alpha=1/4.5)
stc_hat = apply_inverse_operator(evoked, inverse_operator, fwd)
stc_hat.plot(**pp, brain_kwargs=dict(title=solver))

stc_hat.data = contextualize_bd(stc_hat.data, leadfield, steps_per_ep=25, verbose=1)
errors[solver] = corr(stc.data, stc_hat.data)
stc_hat.plot(**pp, brain_kwargs=dict(title=solver))

In [ ]:
import seaborn as sns
import pandas as pd
%matplotlib qt
f, ax = plt.subplots(figsize=(7, 6))
# ax.set_yscale("log")
sns.boxplot(data=pd.DataFrame(errors))